In [1]:
import pandas as pd
import numpy as np
import time,datetime
import gc

In [2]:
def datetime2SecondsFrom1970(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y-%m-%d %H:%M:%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return secondsFrom1970

def seconds2Datetime(seconds_from_1970):
    timeArray = time.localtime(seconds_from_1970)#1970秒数
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    datetime1=datetime.datetime.strptime(otherStyleTime, "%Y-%m-%d %H:%M:%S")
    return str(datetime1)
def datetime2SecondsFrom1970_s(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y%m%d%H%M%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return int(secondsFrom1970)
def self_split_by_comma(str_):
    return str_.split(',')
def self_split_by_long(str_):
    return str_.split('|')

In [9]:
# date_train_extract_list=['0309','0310','0311','0312','0313','0314','0315','0316','0317','0318','0319']
# for date_train_extract in date_train_extract_list:
#     print('The date train set extract :','\n------------------\n2019 -',date_train_extract,'\n------------------')
#     df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
#     df_train_date=df_train_date[df_train_date['create_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
#     df_train_date.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv',index=False)

The date train set extract : 
------------------
2019 - 0309 
------------------
The date train set extract : 
------------------
2019 - 0310 
------------------
The date train set extract : 
------------------
2019 - 0311 
------------------
The date train set extract : 
------------------
2019 - 0312 
------------------
The date train set extract : 
------------------
2019 - 0313 
------------------
The date train set extract : 
------------------
2019 - 0314 
------------------
The date train set extract : 
------------------
2019 - 0315 
------------------
The date train set extract : 
------------------
2019 - 0316 
------------------
The date train set extract : 
------------------
2019 - 0317 
------------------


/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The date train set extract : 
------------------
2019 - 0318 
------------------
The date train set extract : 
------------------
2019 - 0319 
------------------


In [13]:
for day in range(9,20):
    if day <10:
        train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+'030'+str(day)+'_train.csv')
    else:
        train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+'03'+str(day)+'_train.csv')
    print(day,"biger than 0:",train_set_date[train_set_date['exposure_times']>0].shape
          ,"equal 0:",train_set_date[train_set_date['exposure_times']<=0].shape)

9 biger than 0: (4780, 62) equal 0: (9562, 62)
10 biger than 0: (4759, 62) equal 0: (10244, 62)
11 biger than 0: (4637, 62) equal 0: (10856, 62)
12 biger than 0: (4763, 62) equal 0: (10364, 62)
13 biger than 0: (4837, 62) equal 0: (10949, 62)
14 biger than 0: (5090, 62) equal 0: (11247, 62)
15 biger than 0: (5137, 62) equal 0: (12185, 62)
16 biger than 0: (4257, 62) equal 0: (7132, 62)
17 biger than 0: (4346, 62) equal 0: (7560, 62)
18 biger than 0: (3511, 62) equal 0: (6561, 62)
19 biger than 0: (4401, 62) equal 0: (8991, 62)


/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
for day in range(9,20):
    if day <10:
        train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+'030'+str(day)+'_feature1phase_train.csv')
    else:
        train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+'03'+str(day)+'_feature1phase_train.csv')
    print(day,"biger than 0:",train_set_date[train_set_date['exposure_times']>0].shape
          ,"equal 0:",train_set_date[train_set_date['exposure_times']<=0].shape)


9 biger than 0: (4780, 109) equal 0: (9562, 109)
10 biger than 0: (4759, 109) equal 0: (10244, 109)
11 biger than 0: (4637, 109) equal 0: (10856, 109)
12 biger than 0: (4763, 109) equal 0: (10364, 109)
13 biger than 0: (4837, 109) equal 0: (10949, 109)
14 biger than 0: (5090, 109) equal 0: (11247, 109)
15 biger than 0: (5137, 109) equal 0: (12185, 109)
16 biger than 0: (4257, 109) equal 0: (7132, 109)
17 biger than 0: (4346, 109) equal 0: (7560, 109)
18 biger than 0: (3511, 109) equal 0: (6561, 109)
19 biger than 0: (4401, 109) equal 0: (8991, 109)


In [3]:
def get_feature_phase1(date_train_extract,week_day):
    id_type_meanvar_value_dict={'ad_id_bid_mean': 94.72470632815461, 'ad_id_bid_var': 2479.927053140097, 'ad_id_pctr_mean': 7.456589060308556, 'ad_id_pctr_var': 34.23330065649351, 'ad_id_quality_ecpm_mean': 147.76, 'ad_id_quality_ecpm_var': 14000.430519787134, 'ad_id_total_ecpm_mean': 1077.9321328671329, 'ad_id_total_ecpm_var': 842624.7463645847, 'ad_id_history_exposure_times': 19.0, 'account_id_bid_mean': 100.0, 'account_id_bid_var': 1038.6994609164421, 'account_id_pctr_mean': 5.983306693815211, 'account_id_pctr_var': 44.829312856614244, 'account_id_quality_ecpm_mean': 118.35692427790775, 'account_id_quality_ecpm_var': 18034.33437609951, 'account_id_total_ecpm_mean': 723.3500251926495, 'account_id_total_ecpm_var': 697526.679498976, 'account_id_history_exposure_times': 100.0, 'commodity_id_bid_mean': 100.0, 'commodity_id_bid_var': 302.5615384615385, 'commodity_id_pctr_mean': 4.063576923076923, 'commodity_id_pctr_var': 20.48011833986928, 'commodity_id_quality_ecpm_mean': 80.2, 'commodity_id_quality_ecpm_var': 8145.380440080895, 'commodity_id_total_ecpm_mean': 473.3102156862746, 'commodity_id_total_ecpm_var': 271345.84545676154, 'commodity_id_history_exposure_times': 28.0, 'commodity_type_bid_mean': 222.34001000795175, 'commodity_type_bid_var': 16547613.501672428, 'commodity_type_pctr_mean': 16.06287553616584, 'commodity_type_pctr_var': 545.5364423130397, 'commodity_type_quality_ecpm_mean': 389.4260059297535, 'commodity_type_quality_ecpm_var': 1394784.0457270064, 'commodity_type_total_ecpm_mean': 1422.224997028004, 'commodity_type_total_ecpm_var': 7992557.585475998, 'commodity_type_history_exposure_times': 1802431.5, 'trades_id_bid_mean': 118.37847514324936, 'trades_id_bid_var': 44325.1534180594, 'trades_id_pctr_mean': 11.876398353191423, 'trades_id_pctr_var': 362.578142419821, 'trades_id_quality_ecpm_mean': 241.28115127897505, 'trades_id_quality_ecpm_var': 162692.87596998204, 'trades_id_total_ecpm_mean': 1450.9731769679152, 'trades_id_total_ecpm_var': 6590413.504289803, 'trades_id_history_exposure_times': 32880.0}
    print('load exposure data\n......')
    df_totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
    df_totalExposureLog=df_totalExposureLog[df_totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
    print('load train source data')
    df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
    df_train_date['ad_trades_id']=df_train_date['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    
    print('exposurelog nums:',df_totalExposureLog.shape,'train_set nums:',df_train_date.shape)
    print('after ad_id merge:',"biger than 0:",df_train_date[df_train_date['exposure_times']>0].shape
          ,"equal 0:",df_train_date[df_train_date['exposure_times']<=0].shape)
    print('load data finished')
    print('ad_id group bid..pctr mean var start')
    #ad_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['exposure_ad_id'])['bid','pctr','quality_ecpm'
                                                                                     ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['exposure_ad_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'ad_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['ad_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['ad_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['ad_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['ad_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['ad_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['ad_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['ad_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['ad_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_adid_meanvar=pd.merge(df_train_date,df_attribute_mean_var,left_on='ad_id',right_on='exposure_ad_id',how='left')
    print('after ad_id merge:',"biger than 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']<=0].shape)
    df_train_set_date_feature_adid_meanvar.fillna(-8,inplace=True)
    
    print('ad_id mean ==-8 nums:')
    print(df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0].shape)
    df_train_set_date_feature_adid_meanvar['ad_id_bid_mean'][df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0]=id_type_meanvar_value_dict['ad_id_bid_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_bid_var'][df_train_set_date_feature_adid_meanvar['ad_id_bid_var']<0]=id_type_meanvar_value_dict['ad_id_bid_var']
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean']<0]=id_type_meanvar_value_dict['ad_id_pctr_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_var'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_var']<0]=id_type_meanvar_value_dict['ad_id_pctr_var']
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean']<0]=id_type_meanvar_value_dict['ad_id_quality_ecpm_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var']<0]=id_type_meanvar_value_dict['ad_id_quality_ecpm_var']
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean']<0]=id_type_meanvar_value_dict['ad_id_total_ecpm_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var']<0]=id_type_meanvar_value_dict['ad_id_total_ecpm_var']
    df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times'][df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times']<0]=id_type_meanvar_value_dict['ad_id_history_exposure_times']
    
    print('load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted')
    gc.collect()
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                                 names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                                 ])
    df_temp=ad_static_feature[['ad_id','ad_account_id','commodity_id','commodity_type','ad_trades_id']]
    df_totalExposureLog=pd.merge(df_totalExposureLog,df_temp,left_on='exposure_ad_id',right_on='ad_id',how='left')
    del df_temp
    gc.collect()
    
    print('account_id group bid..pctr mean var start')
    #account_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_account_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_account_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'account_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['account_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['account_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['account_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['account_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['account_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['account_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['account_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['account_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_account_id',how='left')
    df_train_set_date_feature_accountid_adid_meanvar.fillna(-8,inplace=True)
    print('after account_id merge:',"biger than 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']<=0].shape)
    print('account_id mean ==-8 nums:')
    print(df_train_set_date_feature_accountid_adid_meanvar['ad_account_id'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0].shape)
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_bid_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_bid_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_pctr_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_pctr_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_quality_ecpm_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_quality_ecpm_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_total_ecpm_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_total_ecpm_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times'][df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['account_id_history_exposure_times']
    
    gc.collect()
    print('commodity_id group bid..pctr mean var start')
    # 'commodity_id'
    df_totalExposureLog.fillna(-2,inplace=True)
    df_totalExposureLog['commodity_id']=df_totalExposureLog['commodity_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_id',how='left')
    df_train_set_date_feature_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_bid_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_bid_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_pctr_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_pctr_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_quality_ecpm_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_quality_ecpm_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_total_ecpm_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_total_ecpm_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['commodity_id_history_exposure_times']
    
    print('commodity_type group bid..pctr mean var start')
    #'commodity_type'
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_type'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_type'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_type_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_type_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_type_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_type_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_type_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_type_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_type_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_type_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_type_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_type',how='left')
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_bid_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_bid_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_pctr_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_pctr_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_quality_ecpm_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_quality_ecpm_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_total_ecpm_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_total_ecpm_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['commodity_type_history_exposure_times']
    
    print('ad_trades_id group bid..pctr mean var start')
    #'ad_trades_id'
    df_totalExposureLog['ad_trades_id']=df_totalExposureLog['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_trades_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_trades_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'trades_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['trades_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['trades_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['trades_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['trades_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['trades_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['trades_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['trades_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['trades_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_trades_id',how='left')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_bid_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_bid_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_pctr_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_pctr_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_quality_ecpm_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_quality_ecpm_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_total_ecpm_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_total_ecpm_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['trades_id_history_exposure_times']
    
    print('week day')
    #weekday
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['week_day']=week_day
    print('ad_size one hot get dummies')
    #ad_size
    df_ad_size_onehot=pd.get_dummies(df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['ad_size'])
    rename_dict={}
    for ad_size_name in df_ad_size_onehot.columns:
        rename_dict[ad_size_name]='ad_size_'+str(ad_size_name)
    df_ad_size_onehot.rename(columns=rename_dict,inplace=True)
    df_train_set_date_feature_adsize_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.concat([df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar,df_ad_size_onehot],axis=1)

    print('write to persistent disk')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv',index=False)
    print('feature phrse 1 train finished')


In [4]:
date_train_extract_list=['0309','0310','0311','0312','0313','0314','0315','0316','0317','0318','0319']
week_day_list=[6,0,1,2,3,4,5,6,0,1,2]
for date_train_extract,week_day in zip(date_train_extract_list,week_day_list):
    print('The date train set extract :','\n------------------\n2019 -',date_train_extract,'\n------------------')
    get_feature_phase1(date_train_extract,week_day)

The date train set extract : 
------------------
2019 - 0309 
------------------
load exposure data
......
load train source data
exposurelog nums: (68725180, 10) train_set nums: (14342, 62)
after ad_id merge: biger than 0: (4780, 62) equal 0: (9562, 62)
load data finished
ad_id group bid..pctr mean var start
after ad_id merge: biger than 0: (4780, 72) equal 0: (9562, 72)
ad_id mean ==-8 nums:
(2318, 72)


/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is tr

load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted


/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


account_id group bid..pctr mean var start
after account_id merge: biger than 0: (4780, 81) equal 0: (9562, 81)
account_id mean ==-8 nums:
(773,)


...

ad_trades_id group bid..pctr mean var start


/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value i

week day
ad_size one hot get dummies
write to persistent disk
feature phrse 1 train finished
The date train set extract : 
------------------
2019 - 0310 
------------------
load exposure data
......
load train source data
exposurelog nums: (72405492, 10) train_set nums: (15003, 62)
after ad_id merge: biger than 0: (4759, 62) equal 0: (10244, 62)
load data finished
ad_id group bid..pctr mean var start
after ad_id merge: biger than 0: (4759, 72) equal 0: (10244, 72)
ad_id mean ==-8 nums:
(2197, 72)
load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted
account_id group bid..pctr mean var start
after account_id merge: biger than 0: (4759, 81) equal 0: (10244, 81)
account_id mean ==-8 nums:
(717,)
commodity_id group bid..pctr mean var start
commodity_type group bid..pctr mean var start
ad_trades_id group bid..pctr mean var start
week day
ad_size one hot get dummies
write to persistent disk
feature phrse 1 train finished
The

/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


exposurelog nums: (88226345, 10) train_set nums: (17322, 62)
after ad_id merge: biger than 0: (5137, 62) equal 0: (12185, 62)
load data finished
ad_id group bid..pctr mean var start
after ad_id merge: biger than 0: (5137, 72) equal 0: (12185, 72)
ad_id mean ==-8 nums:
(2627, 72)
load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted
account_id group bid..pctr mean var start
after account_id merge: biger than 0: (5137, 81) equal 0: (12185, 81)
account_id mean ==-8 nums:
(895,)
commodity_id group bid..pctr mean var start
commodity_type group bid..pctr mean var start
ad_trades_id group bid..pctr mean var start
week day
ad_size one hot get dummies
write to persistent disk
feature phrse 1 train finished
The date train set extract : 
------------------
2019 - 0316 
------------------
load exposure data
......
load train source data
exposurelog nums: (92176148, 10) train_set nums: (11389, 62)
after ad_id merge: biger than 0: (4

In [ ]:
def get_feature_phase1_original(date_train_extract,week_day):
    id_type_meanvar_dict={'ad_id_bid_mean': 94.72470632815461, 'ad_id_bid_var': 2479.927053140097, 'ad_id_pctr_mean': 7.456589060308556, 'ad_id_pctr_var': 34.23330065649351, 'ad_id_quality_ecpm_mean': 147.76, 'ad_id_quality_ecpm_var': 14000.430519787134, 'ad_id_total_ecpm_mean': 1077.9321328671329, 'ad_id_total_ecpm_var': 842624.7463645847, 'account_id_bid_mean': 100.0, 'account_id_bid_var': 1038.6994609164421, 'account_id_pctr_mean': 5.983306693815211, 'account_id_pctr_var': 44.829312856614244, 'account_id_quality_ecpm_mean': 118.35692427790775, 'account_id_quality_ecpm_var': 18034.33437609951, 'account_id_total_ecpm_mean': 723.3500251926495, 'account_id_total_ecpm_var': 697526.679498976, 'commodity_id_bid_mean': 100.0, 'commodity_id_bid_var': 302.5615384615385, 'commodity_id_pctr_mean': 4.063576923076923, 'commodity_id_pctr_var': 20.48011833986928, 'commodity_id_quality_ecpm_mean': 80.2, 'commodity_id_quality_ecpm_var': 8145.380440080895, 'commodity_id_total_ecpm_mean': 473.3102156862746, 'commodity_id_total_ecpm_var': 271345.84545676154, 'commodity_type_bid_mean': 222.34001000795175, 'commodity_type_bid_var': 16547613.501672428, 'commodity_type_pctr_mean': 16.06287553616584, 'commodity_type_pctr_var': 545.5364423130397, 'commodity_type_quality_ecpm_mean': 389.4260059297535, 'commodity_type_quality_ecpm_var': 1394784.0457270064, 'commodity_type_total_ecpm_mean': 1422.224997028004, 'commodity_type_total_ecpm_var': 7992557.585475998, 'trades_id_bid_mean': 118.37847514324936, 'trades_id_bid_var': 44325.1534180594, 'trades_id_pctr_mean': 11.876398353191423, 'trades_id_pctr_var': 362.578142419821, 'trades_id_quality_ecpm_mean': 241.28115127897505, 'trades_id_quality_ecpm_var': 162692.87596998204, 'trades_id_total_ecpm_mean': 1450.9731769679152, 'trades_id_total_ecpm_var': 6590413.504289803}
    print('load exposure data\n......')
    df_totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
    df_totalExposureLog=df_totalExposureLog[df_totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
    print('load train source data')
    df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
    df_train_date['ad_trades_id']=df_train_date['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    print('load data finished')
    print('ad_id group bid..pctr mean var start')
    #ad_id
    df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['exposure_ad_id'])['bid','pctr','quality_ecpm'
                                                                                     ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['exposure_ad_id'])['bid'].size().reset_index()
    df_ad_id_attribute_mean_var.rename(columns={'bid':'ad_id_history_exposure_times'},inplace=True)

    df_ad_id_attribute_mean_var['ad_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
    df_ad_id_attribute_mean_var['ad_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
    df_ad_id_attribute_mean_var['ad_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
    df_ad_id_attribute_mean_var['ad_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
    df_ad_id_attribute_mean_var['ad_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_ad_id_attribute_mean_var['ad_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
    df_ad_id_attribute_mean_var['ad_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
    df_ad_id_attribute_mean_var['ad_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
    df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

    df_test_set_date_feature_adid_meanvar=pd.merge(df_train_date,df_ad_id_attribute_mean_var,left_on='ad_id',right_on='exposure_ad_id',how='left')

    df_test_set_date_feature_adid_meanvar.fillna(-8,inplace=True)
    
    print('ad_id mean ==-8 nums:')
    print(df_test_set_date_feature_adid_meanvar['ad_id'][df_test_set_date_feature_adid_meanvar['ad_id_bid_mean']<0].shape)
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_bid_mean']<0]=id_type_meanvar_dict['ad_id_bid_mean']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_bid_var']<0]=id_type_meanvar_dict['ad_id_bid_var']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_pctr_mean']<0]=id_type_meanvar_dict['ad_id_pctr_mean']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_pctr_var']<0]=id_type_meanvar_dict['ad_id_pctr_var']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean']<0]=id_type_meanvar_dict['ad_id_quality_ecpm_mean']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var']<0]=id_type_meanvar_dict['ad_id_quality_ecpm_var']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean']<0]=id_type_meanvar_dict['ad_id_total_ecpm_mean']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_total_ecpm_var']<0]=id_type_meanvar_dict['ad_id_total_ecpm_var']
    df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_history_exposure_times']<0]=id_type_meanvar_dict['ad_id_history_exposure_times'].median()
    
    print('load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted')
    gc.collect()
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                                 names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                                 ])
    df_temp=ad_static_feature[['ad_id','ad_account_id','commodity_id','commodity_type','ad_trades_id']]
    df_totalExposureLog=pd.merge(df_totalExposureLog,df_temp,left_on='exposure_ad_id',right_on='ad_id',how='left')
    del df_temp
    gc.collect()
    
    print('account_id group bid..pctr mean var start')
    #account_id
    df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_account_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['ad_account_id'])['bid'].size().reset_index()
    df_ad_id_attribute_mean_var.rename(columns={'bid':'account_id_history_exposure_times'},inplace=True)

    df_ad_id_attribute_mean_var['account_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
    df_ad_id_attribute_mean_var['account_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
    df_ad_id_attribute_mean_var['account_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
    df_ad_id_attribute_mean_var['account_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
    df_ad_id_attribute_mean_var['account_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_ad_id_attribute_mean_var['account_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
    df_ad_id_attribute_mean_var['account_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
    df_ad_id_attribute_mean_var['account_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
    df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

    df_test_set_date_feature_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_adid_meanvar,
                                                              df_ad_id_attribute_mean_var,on='ad_account_id',how='left')
    df_test_set_date_feature_accountid_adid_meanvar.fillna(-8,inplace=True)
    print('account_id mean ==-8 nums:')
    print(df_test_set_date_feature_accountid_adid_meanvar['account_id'][df_test_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0].shape)
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0
                                                   ]=id_type_meanvar_dict['account_id_bid_mean']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_bid_var']<0
                                                   ]=id_type_meanvar_dict['account_id_bid_var']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean']<0
                                                   ]=id_type_meanvar_dict['account_id_pctr_mean']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_pctr_var']<0
                                                   ]=id_type_meanvar_dict['account_id_pctr_var']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['account_id_quality_ecpm_mean']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['account_id_quality_ecpm_var']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['account_id_total_ecpm_mean']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['account_id_total_ecpm_var']
    df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times']<0
                                               ]=id_type_meanvar_dict['account_id_history_exposure_times']
    
    gc.collect()
    print('commodity_id group bid..pctr mean var start')
    # 'commodity_id'
    df_totalExposureLog.fillna(-2,inplace=True)
    df_totalExposureLog['commodity_id']=df_totalExposureLog['commodity_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['commodity_id'])['bid'].size().reset_index()
    df_ad_id_attribute_mean_var.rename(columns={'bid':'commodity_id_history_exposure_times'},inplace=True)

    df_ad_id_attribute_mean_var['commodity_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
    df_ad_id_attribute_mean_var['commodity_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
    df_ad_id_attribute_mean_var['commodity_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
    df_ad_id_attribute_mean_var['commodity_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
    df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
    df_ad_id_attribute_mean_var['commodity_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
    df_ad_id_attribute_mean_var['commodity_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
    df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

    df_test_set_date_feature_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_accountid_adid_meanvar,
                                                              df_ad_id_attribute_mean_var,on='commodity_id',how='left')
    df_test_set_date_feature_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_id_bid_mean']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var']<0
                                                   ]=id_type_meanvar_dict['commodity_id_bid_var']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_id_pctr_mean']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var']<0
                                                   ]=id_type_meanvar_dict['commodity_id_pctr_var']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_id_quality_ecpm_mean']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['commodity_id_quality_ecpm_var']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_id_total_ecpm_mean']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['commodity_id_total_ecpm_var']
    df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times']<0
                                               ]=id_type_meanvar_dict['commodity_id_history_exposure_times']
    
    print('commodity_type group bid..pctr mean var start')
    #'commodity_type'
    df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_type'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['commodity_type'])['bid'].size().reset_index()
    df_ad_id_attribute_mean_var.rename(columns={'bid':'commodity_type_history_exposure_times'},inplace=True)

    df_ad_id_attribute_mean_var['commodity_type_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
    df_ad_id_attribute_mean_var['commodity_type_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
    df_ad_id_attribute_mean_var['commodity_type_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
    df_ad_id_attribute_mean_var['commodity_type_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
    df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
    df_ad_id_attribute_mean_var['commodity_type_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
    df_ad_id_attribute_mean_var['commodity_type_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
    df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_commodityid_accountid_adid_meanvar,
                                                              df_ad_id_attribute_mean_var,on='commodity_type',how='left')
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_type_bid_mean']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var']<0
                                                   ]=id_type_meanvar_dict['commodity_type_bid_var']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_type_pctr_mean']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var']<0
                                                   ]=id_type_meanvar_dict['commodity_type_pctr_var']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_type_quality_ecpm_mean']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['commodity_type_quality_ecpm_var']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['commodity_type_total_ecpm_mean']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['commodity_type_total_ecpm_var']
    df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times']<0
                                               ]=id_type_meanvar_dict['commodity_type_history_exposure_times']
    
    
    print('ad_trades_id group bid..pctr mean var start')
    #'ad_trades_id'
    df_totalExposureLog['ad_trades_id']=df_totalExposureLog['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_trades_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['ad_trades_id'])['bid'].size().reset_index()
    df_ad_id_attribute_mean_var.rename(columns={'bid':'trades_id_history_exposure_times'},inplace=True)

    df_ad_id_attribute_mean_var['trades_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
    df_ad_id_attribute_mean_var['trades_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
    df_ad_id_attribute_mean_var['trades_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
    df_ad_id_attribute_mean_var['trades_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
    df_ad_id_attribute_mean_var['trades_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_ad_id_attribute_mean_var['trades_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
    df_ad_id_attribute_mean_var['trades_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
    df_ad_id_attribute_mean_var['trades_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
    df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar,
                                                              df_ad_id_attribute_mean_var,on='ad_trades_id',how='left')
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean']<0
                                                   ]=id_type_meanvar_dict['trades_id_bid_mean']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var']<0
                                                   ]=id_type_meanvar_dict['trades_id_bid_var']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean']<0
                                                   ]=id_type_meanvar_dict['trades_id_pctr_mean']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var']<0
                                                   ]=id_type_meanvar_dict['trades_id_pctr_var']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['trades_id_quality_ecpm_mean']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['trades_id_quality_ecpm_var']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_dict['trades_id_total_ecpm_mean']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_dict['trades_id_total_ecpm_var']
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times']<0
                                               ]=id_type_meanvar_dict['trades_id_history_exposure_times']
    
    print('week day')
    #weekday
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['week_day']=week_day
    print('ad_size one hot get dummies')
    #ad_size
    df_ad_size_onehot=pd.get_dummies(df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['ad_size'])
    rename_dict={}
    for ad_size_name in df_ad_size_onehot.columns:
        rename_dict[ad_size_name]='ad_size_'+str(ad_size_name)
    df_ad_size_onehot.rename(columns=rename_dict,inplace=True)
    df_test_set_date_feature_adsize_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.concat([df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar,df_ad_size_onehot],axis=1)

    print('write to persistent disk')
    df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv.csv',index=False)
    print('feature phrse 1 test finished')


In [ ]:
date_train_extract='0309'
date_before_day_train_extract='0308'
week_day=5

In [ ]:
df_totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
df_totalExposureLog=df_totalExposureLog[df_totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
df_train_date['ad_trades_id']=df_train_date['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))

In [ ]:
print('ad_id group bid..pctr mean var start')
#ad_id
df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['exposure_ad_id'])['bid','pctr','quality_ecpm'
                                                                                 ,'total_ecpm'].aggregate(['mean','var']).reset_index()
df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['exposure_ad_id'])['bid'].size().reset_index()
df_ad_id_attribute_mean_var.rename(columns={'bid':'ad_id_history_exposure_times'},inplace=True)

df_ad_id_attribute_mean_var['ad_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
df_ad_id_attribute_mean_var['ad_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
df_ad_id_attribute_mean_var['ad_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
df_ad_id_attribute_mean_var['ad_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
df_ad_id_attribute_mean_var['ad_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
df_ad_id_attribute_mean_var['ad_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
df_ad_id_attribute_mean_var['ad_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
df_ad_id_attribute_mean_var['ad_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

df_test_set_date_feature_adid_meanvar=pd.merge(df_train_date,df_ad_id_attribute_mean_var,left_on='ad_id',right_on='exposure_ad_id',how='left')

df_test_set_date_feature_adid_meanvar.fillna(-8,inplace=True)
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_bid_mean']<0]=df_ad_id_attribute_mean_var['ad_id_bid_mean'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_bid_var']<0]=df_ad_id_attribute_mean_var['ad_id_bid_var'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_pctr_mean']<0]=df_ad_id_attribute_mean_var['ad_id_pctr_mean'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_pctr_var']<0]=df_ad_id_attribute_mean_var['ad_id_pctr_var'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean']<0]=df_ad_id_attribute_mean_var['ad_id_quality_ecpm_mean'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var']<0]=df_ad_id_attribute_mean_var['ad_id_quality_ecpm_var'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean']<0]=df_ad_id_attribute_mean_var['ad_id_total_ecpm_mean'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_total_ecpm_var']<0]=df_ad_id_attribute_mean_var['ad_id_total_ecpm_var'].median()
df_test_set_date_feature_adid_meanvar[df_test_set_date_feature_adid_meanvar['ad_id_history_exposure_times']<0]=df_ad_id_attribute_mean_var['ad_id_history_exposure_times'].median()

In [ ]:
print('load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted')
gc.collect()
ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                             names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                             ])
df_temp=ad_static_feature[['ad_id','ad_account_id','commodity_id','commodity_type','ad_trades_id']]
df_totalExposureLog=pd.merge(df_totalExposureLog,df_temp,left_on='exposure_ad_id',right_on='ad_id',how='left')
del df_temp
gc.collect()

In [ ]:
print('account_id group bid..pctr mean var start')
#account_id
df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_account_id'])['bid','pctr','quality_ecpm'
                            ,'total_ecpm'].aggregate(['mean','var']).reset_index()
df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['ad_account_id'])['bid'].size().reset_index()
df_ad_id_attribute_mean_var.rename(columns={'bid':'account_id_history_exposure_times'},inplace=True)

df_ad_id_attribute_mean_var['account_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
df_ad_id_attribute_mean_var['account_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
df_ad_id_attribute_mean_var['account_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
df_ad_id_attribute_mean_var['account_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
df_ad_id_attribute_mean_var['account_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
df_ad_id_attribute_mean_var['account_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
df_ad_id_attribute_mean_var['account_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
df_ad_id_attribute_mean_var['account_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

df_test_set_date_feature_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_adid_meanvar,
                                                          df_ad_id_attribute_mean_var,on='ad_account_id',how='left')
df_test_set_date_feature_accountid_adid_meanvar.fillna(-8,inplace=True)

df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_bid_mean'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_bid_var']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_bid_var'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_pctr_mean'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_pctr_var']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_pctr_var'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_quality_ecpm_mean'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_quality_ecpm_var'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_total_ecpm_mean'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_total_ecpm_var'].median()
df_test_set_date_feature_accountid_adid_meanvar[df_test_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times']<0
                                               ]=df_ad_id_attribute_mean_var['account_id_history_exposure_times'].median()

In [ ]:
gc.collect()
print('commodity_id group bid..pctr mean var start')
# 'commodity_id'
df_totalExposureLog.fillna(-2,inplace=True)
df_totalExposureLog['commodity_id']=df_totalExposureLog['commodity_id'].apply(lambda x:int(str(x).split(',')[0]))
df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_id'])['bid','pctr','quality_ecpm'
                            ,'total_ecpm'].aggregate(['mean','var']).reset_index()
df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['commodity_id'])['bid'].size().reset_index()
df_ad_id_attribute_mean_var.rename(columns={'bid':'commodity_id_history_exposure_times'},inplace=True)

df_ad_id_attribute_mean_var['commodity_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
df_ad_id_attribute_mean_var['commodity_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
df_ad_id_attribute_mean_var['commodity_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
df_ad_id_attribute_mean_var['commodity_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
df_ad_id_attribute_mean_var['commodity_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
df_ad_id_attribute_mean_var['commodity_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

df_test_set_date_feature_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_accountid_adid_meanvar,
                                                          df_ad_id_attribute_mean_var,on='commodity_id',how='left')
df_test_set_date_feature_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_bid_mean'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_bid_var'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_pctr_mean'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_pctr_var'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_mean'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_quality_ecpm_var'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_total_ecpm_mean'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_total_ecpm_var'].median()
df_test_set_date_feature_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_id_history_exposure_times'].median()

In [ ]:
print('commodity_type group bid..pctr mean var start')
#'commodity_type'
df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_type'])['bid','pctr','quality_ecpm'
                            ,'total_ecpm'].aggregate(['mean','var']).reset_index()
df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['commodity_type'])['bid'].size().reset_index()
df_ad_id_attribute_mean_var.rename(columns={'bid':'commodity_type_history_exposure_times'},inplace=True)

df_ad_id_attribute_mean_var['commodity_type_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
df_ad_id_attribute_mean_var['commodity_type_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
df_ad_id_attribute_mean_var['commodity_type_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
df_ad_id_attribute_mean_var['commodity_type_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
df_ad_id_attribute_mean_var['commodity_type_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
df_ad_id_attribute_mean_var['commodity_type_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_commodityid_accountid_adid_meanvar,
                                                          df_ad_id_attribute_mean_var,on='commodity_type',how='left')
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_bid_mean'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_bid_var'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_pctr_mean'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_pctr_var'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_mean'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_quality_ecpm_var'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_total_ecpm_mean'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_total_ecpm_var'].median()
df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times']<0
                                               ]=df_ad_id_attribute_mean_var['commodity_type_history_exposure_times'].median()

In [ ]:
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean'].head()

In [ ]:
print('ad_trades_id group bid..pctr mean var start')
#'ad_trades_id'
df_totalExposureLog['ad_trades_id']=df_totalExposureLog['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
df_ad_id_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_trades_id'])['bid','pctr','quality_ecpm'
                            ,'total_ecpm'].aggregate(['mean','var']).reset_index()
df_ad_id_attribute_mean_var=df_totalExposureLog.groupby(['ad_trades_id'])['bid'].size().reset_index()
df_ad_id_attribute_mean_var.rename(columns={'bid':'trades_id_history_exposure_times'},inplace=True)

df_ad_id_attribute_mean_var['trades_id_bid_mean']=df_ad_id_attribute_mean_var_temp[('bid','mean')]
df_ad_id_attribute_mean_var['trades_id_bid_var']=df_ad_id_attribute_mean_var_temp[('bid','var')]
df_ad_id_attribute_mean_var['trades_id_pctr_mean']=df_ad_id_attribute_mean_var_temp[('pctr','mean')]
df_ad_id_attribute_mean_var['trades_id_pctr_var']=df_ad_id_attribute_mean_var_temp[('pctr','var')]
df_ad_id_attribute_mean_var['trades_id_quality_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','mean')]
df_ad_id_attribute_mean_var['trades_id_quality_ecpm_var']=df_ad_id_attribute_mean_var_temp[('quality_ecpm','var')]
df_ad_id_attribute_mean_var['trades_id_total_ecpm_mean']=df_ad_id_attribute_mean_var_temp[('total_ecpm','mean')]
df_ad_id_attribute_mean_var['trades_id_total_ecpm_var']=df_ad_id_attribute_mean_var_temp[('total_ecpm','var')]
df_ad_id_attribute_mean_var.fillna(-4,inplace=True)

df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_test_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar,
                                                                                            df_ad_id_attribute_mean_var,on='ad_trades_id',how='left')
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_bid_mean'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_bid_var'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_pctr_mean'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_pctr_var'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_quality_ecpm_mean'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_quality_ecpm_var'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_total_ecpm_mean'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_total_ecpm_var'].median()
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar[df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times']<0
                                               ]=df_ad_id_attribute_mean_var['trades_id_history_exposure_times'].median()

In [ ]:
print('week day')

#weekday
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['week_day']=week_day
print('ad_size one hot get dummies')
#ad_size
df_ad_size_onehot=pd.get_dummies(df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['ad_size'])
rename_dict={}
for ad_size_name in df_ad_size_onehot.columns:
    rename_dict[ad_size_name]='ad_size_'+str(ad_size_name)
df_ad_size_onehot.rename(columns=rename_dict,inplace=True)
df_test_set_date_feature_adsize_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.concat([df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar,df_ad_size_onehot],axis=1)

print('write to persistent disk')
df_test_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv.csv',index=False)
print('feature phrse 1 test finished')
